In [9]:
import pandas as pd


In [ ]:
# --- 1. MEMBACA DAN MEMPERSIAPKAN DATA ---
print("1. Membaca file data...")
try:
    # Ganti nama file jika berbeda
    # df = pd.read_csv("dataset/data_cuaca_historis_bandung.csv") 
    # df = pd.read_csv("dataset/data_cuaca_telkom_MASTER.csv") 
    # df = pd.read_csv("data_cuaca_augmented.csv") 
    df = pd.read_csv("dataset/data_cuaca_augmented_telkom.csv") 
except FileNotFoundError:
    print("Error: File 'data_cuaca_historis_bandung.csv' tidak ditemukan.")
    exit()

1. Membaca file data...
Error: File 'data_cuaca_historis_bandung.csv' tidak ditemukan.


In [11]:
# Mengubah 'timestamp' menjadi tipe data datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

print(f"Data berhasil dibaca. Jumlah baris: {len(df)}")
print("Contoh data:")
print(df.head())

Data berhasil dibaca. Jumlah baris: 643
Contoh data:
            timestamp  suhu  kelembaban  kecepatan_angin  arah_angin  \
0 2025-09-29 16:50:00  25.3          72              1.1         200   
1 2025-09-29 16:55:00  25.3          72              0.8         211   
2 2025-09-29 17:00:00  25.3          72              0.4         219   
3 2025-09-29 17:05:00  25.5          71              0.4         276   
4 2025-09-29 17:10:00  25.5          71              0.7         286   

   tekanan_udara  intensitas_hujan  intensitas_cahaya  target_prediksi  
0         1013.0               0.0             6300.8                0  
1         1013.1               0.0             5351.8                0  
2         1013.2               0.0             4482.6                0  
3         1013.1               0.0             4239.4                0  
4         1013.1               0.0             3579.3                0  


In [ ]:
def tentukan_kelas_prediksi(row, langkah_prediksi):
    # Cek apakah akan hujan dalam 'langkah_prediksi' ke depan
    akan_hujan = row['intensitas_hujan_mendatang'] > 0
    
    if not akan_hujan:
        return 0 # Kelas 0: Tidak Hujan
    
    # Jika akan hujan, tentukan arahnya dari 'arah_angin' saat ini
    arah_angin = row['arah_angin']
    
    if 337.5 <= arah_angin < 360 or 0 <= arah_angin < 22.5:
        return 1 # Utara
    elif 22.5 <= arah_angin < 67.5:
        return 2 # Timur Laut
    elif 67.5 <= arah_angin < 112.5:
        return 3 # Timur
    elif 112.5 <= arah_angin < 157.5:
        return 4 # Tenggara
    elif 157.5 <= arah_angin < 202.5:
        return 5 # Selatan
    elif 202.5 <= arah_angin < 247.5:
        return 6 # Barat Daya
    elif 247.5 <= arah_angin < 292.5:
        return 7 # Barat
    else: # 292.5 <= arah_angin < 337.5
        return 8 # Barat Laut

# --- Di dalam skrip latih_model.py Anda ---
# Ganti blok Feature Engineering Anda dengan ini:

# ... (setelah membaca df)
langkah_prediksi = 15
df['intensitas_hujan_mendatang'] = df['intensitas_hujan'].shift(-langkah_prediksi).rolling(window=langkah_prediksi).max()
df.dropna(inplace=True)

# Buat target multi-kelas baru
df['target_prediksi'] = df.apply(lambda row: tentukan_kelas_prediksi(row, langkah_prediksi), axis=1)

# Hapus kolom bantuan
df.drop(columns=['intensitas_hujan_mendatang'], inplace=True)

# Target (y) sekarang adalah kolom baru ini
y = df['target_prediksi']
# df.sort_values('target_prediksi').reset_index(drop=False)
# Tambahkan baris ini untuk melihat jumlah data per kelas
print("Mengecek distribusi kelas pada variabel y:")
print(y.value_counts())


Mengecek distribusi kelas pada variabel y:
target_prediksi
0    531
2     18
8     16
5     13
6     12
1     10
7      7
4      5
3      2
Name: count, dtype: int64


: 